In [149]:
#############################################################################################################
### Python classifier based on k-NNFP algorithm (k nearest neighbours on features projection)             ###
#############################################################################################################
#
# The fit() function just stores training data as is
# 
# The predict() function uses the k-nnfp algorithm to return the classes of the data passed as a parameter
# by doing a majority voting for the classes found for each neighbour, according to each individual feature.
#
# In other words, it's like doing k-NN on each individual feature with a majority voting at the end
# to determine the class.
#
# The classifier expects classes to be numbered from 0 to n.
# 
# Furthermore, it expects that missing data be of type np.nan in order to avoid them correctly,
# so you should convert your dataset accordingly (i.e. replace ? with np.nan for example)
#
#############################################################################################################

"""
Extract from http://nimbusvault.net/publications/koala/assr/papers/k17is-223.pdf :

K-Nearest Neighbors on Feature Projections algorithm (k-NNFP):
The K-Nearest Neighbors on Feature Projections (k-NNFP) algorithm is an alternative approach of the
state-of-the-art k-NN classification process. This approach is based on extracted votes applied on a set of
classes, which derive after k-NN performance in each feature separately of the unclassified example.
The most important characteristic of this classification algorithm is that the training set is stored as an
equivalent projection set on feature space. In the dataset that is delivered below in Table 1 three training
examples with two features and a class feature per example is presented.

----------------------------------
Training Example  f0  f1  Class
   Example 1      1   6     B
   Example 2      4   3     A
   Example 3     10   5     A
----------------------------------
Table 1. Training Examples
----------------------------------

The k-NNFP algorithm classifies a given unclassified examples as follows. Firstly,
the k-closest training examples are calculated for each feature, and their classes
are stored, separately. In the next step, the sum of each class is calculated in

feature storage space. Finally, the class with the max sum is performed to be the
class of the unclassified example.
In example, given an unclassified example (<3, 2>), where f0 = 3, f1 = 2 and the
label class is unknown, the k-NNFP approach classifies the example to the ‘A’
class for k = 1 and k = 2, as it is presented in Table 2:

-----------------------------------------------------------------
k   f0      f1     Features’ Bag   Class
1  [A]      [A]      [A, A]          A
2 [A, B]  [A, A]  [A, B, A, A]       A
-----------------------------------------------------------------
Table 2. K-NNFP classification for unclassified example (<3, 2>)
-----------------------------------------------------------------

Considering their differences and the similarities, the k-NN approach focuses on
the distances between training dataset and unclassified examples that provide the
outcome classification on feature space, in opposite with the k-NNFP approach,
which is mainly focused on each feature contribution to the classification process
via the majority of feature votes.
"""


In [1]:
from sklearn.base import BaseEstimator, ClassifierMixin, TransformerMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.multiclass import unique_labels
import numpy as np

# Based on a sklearn default classifier template
class kNNFPClassifier(ClassifierMixin, BaseEstimator):

    def __init__(self, n_neighbours=1):
        self.n_neighbours = n_neighbours

    def fit(self, X, y):
        # Check that X and y have correct shape
        # Commented because we do accept NaN values... they are handled later on in the code below
        # Another possibility would be to impute missing values (mean, median, ...)
        
        #X, y = check_X_y(X, y) 
        
        # Store the classes seen during fit
        self.classes_ = unique_labels(y)
        self.X_ = X
        self.y_ = y

        # Return the classifier
        return self
    
    def predict(self, X):
        # Check is fit had been called
        # Commented because we do accept NaN values... they are handled later on in the code below
        
        #check_is_fitted(self, ['X_', 'y_'])

        # Input validation
        # Commented because we do accept NaN values... they are handled later on in the code below.
        
        #X = check_array(X) 
        
        # Array containing the classes found at the end of the algorithm for every tested instance
        # (every line of array passed in parameter)
        found_class = np.zeros(X.shape[0])
        # Iterating on all the lines of X (all the instances to be classified)
        for instance_index in range(X.shape[0]) :
            # To store the classes of the features for the n_neighbours neighbours
            neigbours_features_classes = np.zeros((self.n_neighbours, X.shape[1]))
            # Iterating on X transposed, so on the columns (features) of the train data
            for column in range(len(self.X_.T)) :
                # Initialize the array of the already found neighbours with -1 values
                # (no neighbours at the moment)
                already_found_neighbours_rows = [-1]*self.n_neighbours
                # We want to find n_neighbours neighbours
                for neighbour_index in range(self.n_neighbours) :
                    # For every new neighbours we seek, we initialize the closest value at the max possible
                    # distance (max possible value for a float64) : every new neighbours found has to be
                    # closer than that.
                    closest = np.finfo(np.float64).max
                    # Iterating on all the train data to look for neighbours
                    for row in range(len(self.X_)) :
                        # We only check for lines which have not already been picked as neighbours
                        if row not in already_found_neighbours_rows :
                            # We are only testing the distances if the data exist (not NaN)
                            if (np.isnan(X[instance_index][column]) == False) & (np.isnan(self.X_[row][column]) == False) :
                                # Computes the distance between the current feature from X and the feature of
                                # the train data of the current iteration
                                distance = abs(X[instance_index][column] - self.X_[row][column])
                                # If we find a closer one... 
                                if distance < closest :
                                    # ...we store its value as the closest known
                                    closest = distance
                                    # Store the class of this new closest
                                    neigbours_features_classes[neighbour_index][column] = self.y_[row]
                                    # Mark the row of this new neighbour as already selected
                                    # (we don't want to parse it again in next iterations)
                                    already_found_neighbours_rows[neighbour_index] = row
            # Flatten the class array in order to apply majority voting
            neigbours_features_classes = neigbours_features_classes.flatten()
            # Cast as a numpy array for majority voting
            neigbours_features_classes = np.array(neigbours_features_classes).astype(int)
            # Counting the bins of every class
            counts = np.bincount(neigbours_features_classes)
            # Majority voting to determine the class
            found_class[instance_index] = np.argmax(counts)

        return found_class


In [2]:
# Test on Iris dataset

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error

iris = datasets.load_iris()
irisdata = iris.data
irisclasses = iris.target

# Create train and test datasets to evaluate generalization performance of the model
# No need to stratify here because the dataset is balanced (50 samples for each of the 3 classes)
# random_state fixed (9) for reproductibility
X_train, X_test, y_train, y_test = train_test_split(irisdata, irisclasses, test_size=0.3, shuffle=True, random_state=9)

knnfp_classifier = kNNFPClassifier(15)
knnfp_classifier.fit(X_train, y_train)

scores = cross_val_score(knnfp_classifier, X_train, y_train, cv=5)
print(f"scores cross_val: {scores}")
print(f"Accuracy: {scores.mean()} +/- {scores.std()}*2")

scores cross_val: [0.85714286 0.9047619  0.95238095 0.85714286 1.        ]
Accuracy: 0.9142857142857143 +/- 0.05553287518900288*2


In [3]:
y_pred = knnfp_classifier.predict(X_test)
print(f"MAE: {mean_absolute_error(y_test,y_pred)}")

MAE: 0.0


In [4]:
### Comparison with classic k-NN algorithm

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=15)
knn.fit(X_train, y_train)

scores = cross_val_score(knn, X_train, y_train, cv=5)
print(f"scores cross_val: {scores}")
print(f"Accuracy: {scores.mean()} +/- {scores.std()}*2")

y_pred = knn.predict(X_test)
print(f"MAE: {mean_absolute_error(y_test,y_pred)}")

scores cross_val: [0.95238095 0.9047619  0.95238095 0.85714286 1.        ]
Accuracy: 0.9333333333333332 +/- 0.04856209060564558*2
MAE: 0.022222222222222223


In [5]:
### k-NNFP seems to work slightly better on test set. Restults are rather similar however.

In [5]:
!pip install ucimlrepo

/bin/bash: /home/micmac/anaconda3/envs/keras/lib/libtinfo.so.6: no version information available (required by /bin/bash)
  Obtaining dependency information for ucimlrepo from https://files.pythonhosted.org/packages/85/8b/aab8a1c1344af158feb0b7f13d15ae184bc1e93625cea98d9c783b2e29d4/ucimlrepo-0.0.2-py3-none-any.whl.metadata


In [6]:
### Test on the Wine dataset (UCI) : predict the origin country of wines basing on their chemical composition

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error

from ucimlrepo import fetch_ucirepo

# Fetch dataset 
wine = fetch_ucirepo(id=109) 
  
# Data (as pandas dataframes) 
X = wine.data.features 
y = wine.data.targets 

X = X.to_numpy()
X = np.array(X).astype(float)
y = y.to_numpy()
y = np.array(y).astype(int)
y = y.ravel() #Converts the classes in scikit-learn format (1d list)

# Changes the range of classes to be 0 to n (instead of 1 to n)
y = y - np.array([1]*y.shape[0])

# Random shuffling to avoid a (possible) pre-existing ordering in the data
from sklearn.utils import shuffle
X, y = shuffle(X, y, random_state=9)

# Create train and test datasets to evaluate generalization performance of the model
# Doing stratify here because of imbalanced dataset
# random_state fixed (9) for reproductibility
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=9)

knnfp_classifier = kNNFPClassifier(5)
knnfp_classifier.fit(X_train, y_train)

scores = cross_val_score(knnfp_classifier, X_train, y_train, cv=5)
print(f"scores cross_val: {scores}")
print(f"Accuracy: {scores.mean()} +/- {scores.std()}*2")

y_pred = knnfp_classifier.predict(X_test)
print(f"MAE: {mean_absolute_error(y_test,y_pred)}")

scores cross_val: [0.96551724 1.         0.96428571 0.96428571 0.96428571]
Accuracy: 0.9716748768472907 +/- 0.01417059099866258*2
MAE: 0.05555555555555555


In [7]:
### Comparison with classic k-NN algorithm

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

scores = cross_val_score(knn, X_train, y_train, cv=5)
print(f"scores cross_val: {scores}")
print(f"Accuracy: {scores.mean()} +/- {scores.std()}*2")

y_pred = knn.predict(X_test)
print(f"MAE: {mean_absolute_error(y_test,y_pred)}")

scores cross_val: [0.68965517 0.68965517 0.75       0.57142857 0.67857143]
Accuracy: 0.6758620689655173 +/- 0.05794933694325902*2
MAE: 0.3333333333333333


In [8]:
### On this dataset, k-NNFP works much better than k-NN

In [8]:
### Test on the Dermatology dataset (UCI) : skin disease diagnostic (classification)
import pandas as pd

col_list = [*range(0,34,1)]
data_frame = pd.read_csv("./datasets/DermatologyDataset/dermatology.data", sep=",", header=None, usecols=col_list, encoding ='latin1')

col_list = [34]
classes = pd.read_csv("./datasets/DermatologyDataset/dermatology.data", sep=",", header=None, usecols=col_list, encoding ='latin1')

# Replace missing values with np.nan (which our classifier handles)
data_frame.replace('?',np.nan, inplace=True)

# Converts the dataframe into a numpy array, and stores the classes in a separate np array
dermato_data = data_frame.to_numpy()
dermato_data = np.array(dermato_data).astype(float)
dermato_classes = classes.to_numpy()
dermato_classes = np.array(dermato_classes).astype(int)
dermato_classes = dermato_classes.ravel() #met les classes sous la forme demandée par scikit-learn (liste 1d)

# Changes the range of classes to be 0 to n (instead of 1 to n)
dermato_classes = dermato_classes - np.array([1]*dermato_classes.shape[0])

In [22]:
# Create train and test datasets to evaluate generalization performance of the model
# Doing stratify here because of imbalanced dataset
# random_state fixed (9) for reproductibility
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer

# Random shuffling to avoid a (possible) pre-existing ordering in the data
from sklearn.utils import shuffle
dermato_data, dermato_classes = shuffle(dermato_data, dermato_classes, random_state=0)

X_train, X_test, y_train, y_test = train_test_split(dermato_data, dermato_classes, stratify=dermato_classes, test_size=0.2)

knnfp_classifier = kNNFPClassifier(5)
knnfp_classifier.fit(X_train, y_train)

scores = cross_val_score(knnfp_classifier, X_train, y_train, cv=5)
print(f"scores cross_val: {scores}")
print(f"Accuracy: {scores.mean()} +/- {scores.std()}*2")

scores cross_val: [0.15254237 0.49152542 0.48275862 0.44827586 0.44827586]
Accuracy: 0.40467562828755116 +/- 0.12728941819646267*2


In [21]:
y_pred = knnfp_classifier.predict(X_test)
print(f"MAE: {mean_absolute_error(y_test,y_pred)}")

MAE: 0.7972972972972973


In [23]:
### Comparing with classical k-NN algorithm
### Imputing missing data by the mean

from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(X_train)
X_train_imputed = imp.transform(X_train)
imp.fit(X_test)
X_test_imputed = imp.transform(X_test)

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_imputed, y_train)

scores = cross_val_score(knn, X_train_imputed, y_train, cv=5)
print(f"scores cross_val: {scores}")
print(f"Accuracy: {scores.mean()} +/- {scores.std()}*2")

scores cross_val: [0.83050847 0.86440678 0.96551724 0.86206897 0.87931034]
Accuracy: 0.8803623611922852 +/- 0.04544689049020901*2


In [27]:
y_pred = knn.predict(X_test_imputed)
print(f"MAE: {mean_absolute_error(y_test,y_pred)}")

from sklearn.metrics import accuracy_score
print(f"test accuracy: {accuracy_score(y_test, y_pred)}")


MAE: 0.5405405405405406
test accuracy: 0.7702702702702703


In [195]:
### Bad results for k-NNFP on this dataset...

In [29]:
# Test on the Arrythmia dataset (UCI)

import pandas as pd

col_list = [*range(0,279,1)]
data_frame = pd.read_csv("./datasets/ArrhythmiaDataset/arrhythmia.data", sep=",", header=None, usecols=col_list, encoding ='latin1')

col_list = [279]
classes = pd.read_csv("./datasets/ArrhythmiaDataset/arrhythmia.data", sep=",", header=None, usecols=col_list, encoding ='latin1')

# Replace missing values with np.nan (which our classifier handles)
data_frame.replace('?',np.nan, inplace=True)

# Converts the dataframe into a numpy array, and stores the classes in a separate np array
arrythm_data = data_frame.to_numpy()
arrythm_data = np.array(arrythm_data).astype(float)
arrythm_classes = classes.to_numpy()
arrythm_classes = np.array(arrythm_classes).astype(int)
arrythm_classes = arrythm_classes.ravel() #met les classes sous la forme demandée par scikit-learn (liste 1d)

# Changes the range of classes to be 0 to n (instead of 1 to n)
arrythm_classes = arrythm_classes - np.array([1]*arrythm_classes.shape[0])

In [30]:
# Create train and test datasets to evaluate generalization performance of the model
# No need to stratify here because the dataset is balanced (50 samples for each of the 3 classes)
# random_state fixed (9) for reproductibility
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error

# Random shuffling to avoid a (possible) pre-existing ordering in the data
arrythm_data, arrythm_classes = shuffle(arrythm_data, arrythm_classes, random_state=0)

X_train, X_test, y_train, y_test = train_test_split(arrythm_data, arrythm_classes, test_size=0.3)

knnfp_classifier = kNNFPClassifier(5)
knnfp_classifier.fit(X_train, y_train)

scores = cross_val_score(knnfp_classifier, X_train, y_train, cv=5)
print(f"scores cross_val: {scores}")
print(f"Accuracy: {scores.mean()} +/- {scores.std()}*2")

/home/micmac/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


scores cross_val: [0.5625     0.57142857 0.55555556 0.55555556 0.55555556]
Accuracy: 0.5601190476190476 +/- 0.006261799142087081*2


In [31]:
y_pred = knnfp_classifier.predict(X_test)
print(f"MAE: {mean_absolute_error(y_test,y_pred)}")

from sklearn.metrics import accuracy_score
print(f"test accuracy: {accuracy_score(y_test, y_pred)}")

MAE: 3.1544117647058822
test accuracy: 0.5


In [143]:
### Bad score on this dataset, but there are under-populated classes (1, 2, or 3 members)
### and a majoritary class (more than 200 members)


In [32]:
### Trying imputing missing data by the mean

from sklearn.impute import SimpleImputer

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error

# Random shuffling to avoid a (possible) pre-existing ordering in the data
from sklearn.utils import shuffle
arrythm_data, arrythm_classes = shuffle(arrythm_data, arrythm_classes, random_state=0)

X_train, X_test, y_train, y_test = train_test_split(arrythm_data, arrythm_classes, test_size=0.3)

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(X_train)
X_train_imputed = imp.transform(X_train)
imp.fit(X_test)
X_test_imputed = imp.transform(X_test)

In [33]:
knnfp_classifier = kNNFPClassifier(5)
knnfp_classifier.fit(X_train, y_train)

scores = cross_val_score(knnfp_classifier, X_train_imputed, y_train, cv=5, error_score="raise")
print(f"scores cross_val: {scores}")
print(f"Accuracy: {scores.mean()} +/- {scores.std()}*2")

/home/micmac/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


scores cross_val: [0.0625     0.55555556 0.55555556 0.55555556 0.55555556]
Accuracy: 0.4569444444444445 +/- 0.19722222222222224*2


In [34]:
y_pred = knnfp_classifier.predict(X_test)
print(f"MAE: {mean_absolute_error(y_test,y_pred)}")

from sklearn.metrics import accuracy_score
print(f"test accuracy: {accuracy_score(y_test, y_pred)}")

MAE: 2.639705882352941
test accuracy: 0.5147058823529411


In [1]:
### Not much improvement with imputation either...

/tmp/ipykernel_6253/260256341.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML
